In [25]:
!pip install -Uqqq pip
!pip install -qqq torch==2.1.2
!pip install -qqq causal-conv1d==1.1.1
!pip install -qqq mamba-ssm==1.1.1
!pip install langchain==0.1.1
!pip install qdrant-client==1.7.0
!pip install FlagEmbedding==1.1.8
!pip install huggingface-hub==0.20.2
!pip install sentence-transformers==2.2.2
!pip install transformers==4.36.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flagembedding 1.1.8 requires transformers==4.34.0, but you have transformers 4.36.2 which is incompatible.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
  Using cached transformers-4.34.0-py3-none-any.whl.metadata (121 kB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
Using cached transforme

  Using cached huggingface_hub-0.20.2-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.20.2-py3-none-any.whl (330 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.20.2 which is incompatible.


  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2


  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
  Using cached tokenizers-0.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
Using cached tokenizers-0.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=f1b71e70ee58d9ac885e8e2bff005342adb5fe40f4dd8d8b3eac329214931aad
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [26]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [15]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link



In [31]:
from inspect import cleandoc

import pandas as pd
import torch
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [6]:
def memory_usage() -> pd.DataFrame:
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    return pd.DataFrame(
        [
            {
                "gpu": gpu_stats.name,
                "max_memory_gb": max_memory,
                "used_memory_gb": start_gpu_memory,
            }
        ]
    )


In [7]:
memory_usage()

,gpu,max_memory_gb,used_memory_gb
0,NVIDIA A100-SXM4-40GB,39.564,0.0


In [24]:

MODEL_NAME = "havenhq/mamba-chat"
ANSWER_START = "<|assistant|>\n"
ANSWER_END = "<|endoftext|>"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.eos_token = ANSWER_END
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(
    "BAAI/bge-small-en-v1.5"
).chat_template

model = MambaLMHeadModel.from_pretrained(MODEL_NAME, device=DEVICE, dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
def model_billion_parameters(model, round_digits: int = 2) -> float:
    n_params = sum(p.numel() for p in model.parameters())
    return round(n_params / 1e9, round_digits)


f"{model_billion_parameters(model)}B parameters"

'2.77B parameters'

In [10]:
memory_usage()

,gpu,max_memory_gb,used_memory_gb
0,NVIDIA A100-SXM4-40GB,39.564,5.49


In [11]:
prompt = "What is the meaning of life?"

In [12]:
messages = [{"role": "user", "content": prompt}]
messages

[{'role': 'user', 'content': 'What is the meaning of life?'}]

In [13]:
input_ids = tokenizer.apply_chat_template(
    messages, return_tensors="pt", add_generation_prompt=True
).to(DEVICE)
input_ids

tensor([[   29,    93,  4537, 49651,   187,  1276,   310,   253,  4495,   273,
          1495,    32,     0,   187,    29,    93,   515,  5567, 49651,   187]],
       device='cuda:0')

In [16]:
%%time
outputs = model.generate(
    input_ids=input_ids,
    max_length=1024,
    temperature=0.9,
    top_p=0.7,
    eos_token_id=tokenizer.eos_token_id,
)

CPU times: user 8.71 s, sys: 87.9 ms, total: 8.8 s
Wall time: 11.5 s


In [17]:
response = tokenizer.decode(outputs[0])
print(response)

<|user|>
What is the meaning of life?<|endoftext|>
<|assistant|>
The meaning of life is a philosophical question that has been debated for centuries. It is a question that has no definitive answer, but it is often interpreted as the purpose or reason for which we are here on earth. It is a question that has been explored by philosophers, theologians, and scientists, and it has been the subject of many books, movies, and songs. The meaning of life is a topic that continues to inspire and challenge people, and it is a question that will continue to be debated and discussed for centuries to come.<|endoftext|>


In [18]:
def extract_response(output: str) -> str:
    response_start = output.find(ANSWER_START) + len(ANSWER_START)
    return output[response_start : output.find(ANSWER_END, response_start)]

In [19]:
print(extract_response(response))

The meaning of life is a philosophical question that has been debated for centuries. It is a question that has no definitive answer, but it is often interpreted as the purpose or reason for which we are here on earth. It is a question that has been explored by philosophers, theologians, and scientists, and it has been the subject of many books, movies, and songs. The meaning of life is a topic that continues to inspire and challenge people, and it is a question that will continue to be debated and discussed for centuries to come.


In [20]:
def predict(prompt: str, system_prompt: str = "") -> str:
    messages = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    messages.append({"role": "user", "content": prompt})

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(DEVICE)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=1024,
        temperature=0.9,
        top_p=0.7,
        eos_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0])
    return extract_response(response)

In [21]:
### Text Generation

In [22]:
%%time
print(predict("What are the pros/cons of ChatGPT vs Open Source LLMs?"))

ChatGPT vs Open Source LLMs:

Pros:

1. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

2. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

Cons:

1. ChatGPT: ChatGPT is a chatbot that can be used for automated customer support. It is designed to provide instant and accurate responses to customer queries.

2. Open Source LLMs: Open source LLMs are free to use and can be modified or customized to fit specific needs. They are also available for free download.

3. ChatG

In [23]:
%%time
prompt = cleandoc(
    """
What is the most iconic dish that slavics prepare for Christmas?
"""
)
print(predict(prompt))
print()

The most iconic dish that Slavs prepare for Christmas is the "Slavic Christmas Eve dinner." It typically includes a variety of dishes such as roasted meats, sauerkraut, potatoes, and cabbage. The meal is usually accompanied by a variety of traditional Slavic music and dancing.

CPU times: user 2.77 s, sys: 0 ns, total: 2.77 s
Wall time: 2.77 s


In [28]:
## Data Prparation

In [29]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [32]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [33]:
len(texts)

233

In [34]:
texts[3]

Document(page_content='The dataset, which notes that AI-centered startups are raising more and at higher valuations than other startups, indicates that perhaps the best way to avoid a down round today is to build in the artificial intelligence space.\n\nWhat the data says\n\nPer Carta data relating to the first quarter of the year, seed funding to non-AI startups in the U.S. market that use its services dipped from $1.64 billion to $1.08 billion, or a decline of around 34%. That result is directionally aligned with other data that we’ve seen regarding Q1 2023 venture capital totals; the data points down.', metadata={'source': 'new_articles/05-06-ai-startups-q1-investments.txt'})

In [35]:
## create the DB

In [36]:
def get_index(): #creates and returns an in-memory vector store to be used in the application

    model_name = "BAAI/bge-small-en-v1.5"
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

    embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
        )

    index_from_loader = Qdrant.from_documents(
            texts,
            embeddings,
            location=":memory:",  # Local mode with in-memory storage only
            collection_name="my_documents",
        )

    return index_from_loader #return the index to be cached by the client app

In [37]:
vector_index = get_index()

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [38]:
vector_index

In [39]:
def semantic_search(index, original_prompt): #rag client function

    relevant_prompts = index.similarity_search(original_prompt)

    list_prompts = []
    for i in range(len(relevant_prompts)):
        list_prompts.append(relevant_prompts[i].page_content)

    return list_prompts

['Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”',
 'But Pando has a compelling sales pitch, judging by its momentum. The company counts Fortune 500 manufacturers and r

In [55]:
def predict(prompt: str) -> str:
    selected_prompt = semantic_search(vector_index, prompt)
    selected_prompt = ' , '.join(selected_prompt)
    messages = []

    if selected_prompt:
        messages.append({"role": "system", "content": "Please respond to the original query. If the selected document prompt is relevant and informative, provide a detailed answer based on its content. However, if the selected prompt does not offer useful information or is not applicable, simply state 'No answer found'."})

    messages.append({"role": "user", "content": f"""Original Prompt: {prompt}\n\n
                    Selected Prompt: {selected_prompt}\n\n
                    respond: """})

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(DEVICE)
    outputs = model.generate(
        input_ids=input_ids,
        max_length=1024,
        temperature=0.9,
        top_p=0.7,
        eos_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0])
    return extract_response(response)

In [56]:
predict("How much money did Pando raise?")

'Selected Prompt: How much money did Pando raise?\n\nSelected Answer: $30 million in a Series B round, bringing its total raised to $45 million.'

In [57]:
predict("What is the news about Pando?")

'Selected Prompt: What is the news about Pando?\n\nSelected Response: Pando has raised $30 million in a Series B round, bringing its total raised to $45 million. The startup is led by Nitin Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace. The startup is focused on global logistics and supply chain management through a software-as-a-service platform. Pando has a compelling sales, marketing and delivery capabilities, according to Jayakrishnan. The startup has also tapped existing enterprise users at warehouses, factories, freight yards and ports and expects to expand its customer base. The company is also open to exploring strategic partnerships and acquisitions with this round of funding.'

In [58]:
predict("Who led the round in Pando?")

'Selected Prompt: Who led the round in Pando?\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures, Next47, and Pando.'

In [59]:
predict("What did databricks acquire?")

'Databricks acquired Okera, a data governance platform with a focus on AI, for $30 million. The two companies did not disclose the purchase price. According to Crunchbase, Okera previously raised just under $30 million. Investors include Felicis, Bessemer Venture Partners, Cyber Mentor Fund, ClearSky and Emergent Ventures. Databricks also stated that the acquisition will enable it to expose additional APIs that its own data governance partners will be able to use to provide solutions to their customers.'

In [60]:
predict("What is generative ai?")

"The selected response is:\n\nBird responded that Microsoft is exploring ways to compensate creators whose work is swept up in training data, even if the model training isn't necessarily abusive. However, the company is also exploring ways to allow users to opt out of the training dataset for one AI model, Stable Diffusion v3."